<a href="https://colab.research.google.com/github/daffamaulana03/Fuzzy-Logic/blob/master/code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy
import pandas
import csv

# **BACA DATA**

In [0]:
def readFile():
    data = open("influencers.csv", "r")
    datata = data.readlines()
    
    array = []


    for i in range(1, len(datata)):
      array.append(datata[i].replace('\n','').split(','))
    # print(array[1][1])
    # print(len(array))
    return array

# **FUZZIFIKASI**

Atribut Followers

In [0]:
def fuzzificationFollowers():
  data = readFile()
  followers = []
  # FOLLOWERS

  # Upper	: >70K NOT Upper : <=50K
  # Middle	: 40K<=MID<=50K NOT Mid : <=20K & >70K
  # Bottom	: <=20K NOT Bottom : >30K

  Upper = 70000
  NotUpper = 55000
  Middle1 = 40000
  Middle2 = 50000
  Bottom = 20000
  NotBottom = 30000

  for i in range(len(data)):
    # for j in range(len(data[i])):
    x = int(data[i][1])
    #UPPER
    if (x > Upper):
      Up = 1
    elif (x <= NotUpper):
      Up = 0
    else:
      Up = (x-NotUpper)/(Upper-NotUpper)

    #BOTTOM
    if (x <= Bottom):
      Bot = 1
    elif (x > NotBottom):
      Bot = 0
    else:
      Bot = (NotBottom-x)/(NotBottom-Bottom)

    #MIDDLE
    if ((x > Middle1) & (x <= Middle2)):
      Mid = 1
    elif ((x <= Bottom) or (x > Upper)):
      Mid = 0
    else:
      if (x > Middle2):
        Mid = (Upper-x)/(Upper-Middle2)
      elif (x <= Middle1):
        Mid = (x-Bottom)/(Middle1-Bottom)
    
    fol = []
    foll = []
    fol.append(Up)
    fol.append('upper')
    foll.append(fol)
    fol = []
    fol.append(Mid)
    fol.append('middle')
    foll.append(fol)
    fol = []
    fol.append(Bot)
    fol.append('bottom')
    foll.append(fol)
    fol = []
    followers.append(foll)
  return followers

Atribut Engagement Rate

In [0]:
def fuzzificationEngageRate():
  data = readFile()
  EngageRate = []
  # ENGAGEMENT RATE

  # High	: >7 NOT High : <=5.5
  # Average	: 4<AVG<5 NOT Mid : <=2 & >7
  # Low	: <=2 NOT Low : >3
  High = 7
  NotHigh = 5.5
  Average1 = 4
  Average2 = 5
  Low = 2
  NotLow = 3

  for i in range(len(data)):
    # for j in range(len(data[i])):
    x = float(data[i][2])
    #HIGH
    if (x > High):
      Hi = 1
    elif (x <= NotHigh):
      Hi = 0
    else:
      Hi = (x-NotHigh)/(High-NotHigh)

    #LOW
    if (x <= Low):
      Lo = 1
    elif (x > NotLow):
      Lo = 0
    else:
      Lo = (NotLow-x)/(NotLow-Low)

    #AVERAGE
    if ((x > Average1) & (x <= Average2)):
      Avg = 1
    elif ((x <= Low) or (x > High)):
      Avg = 0
    else:
      if (x > Average2):
        Avg = (High-x)/(High-Average2)
      elif (x <= Average1):
        Avg = (x-Low)/(Average1-Low)
    
    rat = []
    rate = []
    rat.append(Hi)   #HIGH     (indeks0)
    rat.append('high')
    rate.append(rat)
    rat = []
    rat.append(Avg)  #AVERAGE  (indeks1)
    rat.append('average')
    rate.append(rat)
    rat = []
    rat.append(Lo)   #LOW      (indeks2)
    rat.append('low')
    rate.append(rat)
    rat = []
    EngageRate.append(rate)
  
  return EngageRate

# **RULE INFERENSI**

In [0]:
def inferensi():
  # FOLLOWERS	| ENGAGEMENT RATE	|	SCORE
  # Upper		  | High			      | Chosen
  # Upper		  |  Average			  | Chosen
  # Upper		  |  Low			      | Considered
    
  # Middle		|  High			      | Chosen
  # Middle		|  Average			  | Considered
  # Middle		|  Low			      | Rejected

  # Bottom		|  High			      | Considered
  # Bottom		|  Average			  | Rejected
  # Bottom		|  Low			      | Rejected

  followers = fuzzificationFollowers()
  engageRate = fuzzificationEngageRate()
  concs = []

  for i in range(len(followers)):
    rules = [[followers[i][0],engageRate[i][0]],
             [followers[i][0],engageRate[i][1]],
             [followers[i][0],engageRate[i][2]],
             [followers[i][1],engageRate[i][0]],
             [followers[i][1],engageRate[i][1]],
             [followers[i][1],engageRate[i][2]],
             [followers[i][2],engageRate[i][0]],
             [followers[i][2],engageRate[i][1]],
             [followers[i][2],engageRate[i][2]]]
    
    Chosen = []
    Considered = []
    Rejected = []
    for z in range(len(rules)):
      if (rules[z][0][1] == 'upper'):
        if (rules[z][1][1] == 'high'):
          Chosen.append(z)
        elif (rules[z][1][1] == 'average'):
          Chosen.append(z)
        elif (rules[z][1][1] == 'low'):
          Considered.append(z)

      elif (rules[z][0][1] == 'middle'):
        if (rules[z][1][1] == 'high'):
          Chosen.append(z)
        elif (rules[z][1][1] == 'average'):
          Considered.append(z)
        elif (rules[z][1][1] == 'low'):
          Rejected.append(z)

      elif (rules[z][0][1] == 'bottom'):
        if (rules[z][1][1] == 'high'):
          Considered.append(z)
        elif (rules[z][1][1] == 'average'):
          Rejected.append(z)
        elif (rules[z][1][1] == 'low'):
          Rejected.append(z)
    
    for x in range(len(rules)):
      minn = min(rules[x])
      rules[x].append(minn)

    ch = max(rules[Chosen[0]][2][0],rules[Chosen[1]][2][0],rules[Chosen[2]][2][0])
    co = max(rules[Considered[0]][2][0],rules[Considered[1]][2][0],rules[Considered[2]][2][0])
    re = max(rules[Rejected[0]][2][0],rules[Rejected[1]][2][0],rules[Rejected[2]][2][0])
    conc = []
    conc.append(ch)
    conc.append(co)
    conc.append(re)
    concs.append(conc)
  # print("Ch",Chosen)
  # print("Co",Considered)
  # print("Re",Rejected)


    
  # print("HASIL Inferensi")
  # for k in range(len(concs)):
  #     print(k+1,concs[k])
  return concs

# **DEFUZZIFIKASI**

Using Takagi-Sugeno-style (constant defuzzification)

In [0]:
def defuzzification():
  inf = inferensi()
  hasilDef = []

  Chosen = 100
  Considered = 70
  Rejected = 50

  for i in range(len(inf)):
    Z = (Chosen*inf[i][0])+(Considered*inf[i][1])+(Rejected*inf[i][2])
    hasilDef.append(Z/sum(inf[i]))

  # for k in range(len(hasilDef)):
  #   print(k+1,hasilDef[k])

  return hasilDef

# **SORTING**

Menentukan 20 Influencer Terbaik

In [0]:
def BestInfluencer():
  defuzzi = defuzzification()
  data = readFile()
  
  bestt = []

  ngurut = numpy.argsort(defuzzi)[::-1]
  for i in range(20):
    best = []
    best.append(data[ngurut[i]][0])
    bestt.append(best)
    # print(defuzzi[ngurut[i]])
    # print(ngurut[i],defuzzi[ngurut[i]])
  # for k in range(20):
  #   bestt[k].insert(0,k+1)
  # for j in range(20):
  #   print(j+1,data[ngurut[j]])
  return bestt

# **WRITE TO CSV**

In [0]:
def writeCSV():
  best = BestInfluencer()
  f = open('chosen.csv', 'w')
  w = csv.writer(f)
  w.writerow(('Nomor Record',))
  w.writerows(best)

writeCSV()